## Imports e configs

In [8]:
# imports
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen

# pandas config
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:,.2f}'.format

# seed reproducibilidade
myseed = 484

## Abrindo os dados

In [9]:
# pegando os dados do github e aprindo o json
data_url = "https://github.com/sthemonica/alura-voz/blob/main/Dados/Telco-Customer-Churn.json?raw=true"
response = urlopen(data_url)
data_json = json.loads(response.read())

# carregando o dataframe
df = pd.json_normalize(data_json, max_level=2, sep='_')

# removendo o prefixo
df.columns = ['_'.join(cols.split('_')[1:]) if len(cols.split('_')) > 1 else cols for cols in df.columns]

# mostrando o cabecalho
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4


## DataFrame de Verificacao de Dados

Funcao para criacao do df check

In [4]:
def df_check(df):
    
    # criando dataframe para verificacao
    check = pd.DataFrame({'name': df.columns})

    # verificando quantos valores unicos existem em cada  (numero maximo possivel 7267)
    check['nunique'] = df.nunique().values

    # verificando quantidades de nulos
    check['isnull'] = df.isnull().sum().values

    # verificando espacos faltantes
    temp_series1 = df[df.select_dtypes('object').columns].apply(lambda x: x.str.strip().isin(['']).sum()) # colunas str
    temp_series2 = df.select_dtypes(['float', 'integer']).isnull().sum() # colunas numericos
    temp_series = pd.concat([temp_series1, temp_series2])
    temp_series.name = 'blank'
    check = check.merge(temp_series, how='left', left_on='name', right_index=True)

    # verificando data type
    check['dtypes'] = df.dtypes.values

    # vericando quais os valores unicos para colunas com 5 ou menos valores unicos
    check['unique'] = df.apply(lambda x: x.unique() if x.nunique() <= 5 else '-').values

    return check

In [12]:
check = df_check(df)
check

,name,nunique,isnull,blank,dtypes,unique
0,customerID,7267,0,0,object,-
1,Churn,3,0,224,object,"[No, Yes, ]"
2,gender,2,0,0,object,"[Female, Male]"
3,SeniorCitizen,2,0,0,int64,"[0, 1]"
4,Partner,2,0,0,object,"[Yes, No]"
5,Dependents,2,0,0,object,"[Yes, No]"
6,tenure,73,0,0,int64,-
7,PhoneService,2,0,0,object,"[Yes, No]"
8,MultipleLines,3,0,0,object,"[No, Yes, No phone service]"
9,InternetService,3,0,0,object,"[DSL, Fiber optic, No]"


        Aqui eh possivel notar que 2 colunas do data frama apresentam dados em branco ('blank')

Dicionario de informacao das colunas

In [11]:
# dicionario informacao das colunas
dict_info_cols = { 
    'customerID': 'número de identificação único de cada cliente',
    'Churn': 'se o cliente deixou ou não a empresa',
    'gender': 'gênero (masculino e feminino)' ,
    'SeniorCitizen': 'informação sobre um cliente ter ou não idade igual ou maior que 65 anos',
    'Partner':  'se o cliente possui ou não um parceiro ou parceira',
    'Dependents': 'se o cliente possui ou não dependentes',
    'tenure':  'meses de contrato do cliente',
    'PhoneService': 'assinatura de serviço telefônico' ,
    'MultipleLines': 'assisnatura de mais de uma linha de telefone' ,
    'InternetService': 'assinatura de um provedor internet' ,
    'OnlineSecurity': 'assinatura adicional de segurança online' ,
    'OnlineBackup': 'assinatura adicional de backup online' ,
    'DeviceProtection': 'assinatura adicional de proteção no dispositivo' ,
    'TechSupport': 'assinatura adicional de suporte técnico, menos tempo de espera',
    'StreamingTV': 'assinatura de TV a cabo',
    'StreamingMovies': 'assinatura de streaming de filmes' ,
    'Contract': 'tipo de contrato',
    'PaperlessBilling': 'se o cliente prefere receber online a fatura',
    'PaymentMethod': 'forma de pagamento',
    'Charges_Monthly': 'total de todos os serviços do cliente por mês',
    'Charges_Total': 'total gasto pelo cliente'
    }